*code review*

In [1]:
import hashlib
import time
import random
def hash_file(file_path):
    # Open the file in binary mode
    with open(file_path, "rb") as file:
        # Read the entire file content
        file_content = file.read()
        # Calculate the SHA-256 hash of the file content
        hash_value = hashlib.sha256(file_content).hexdigest()
    return hash_value

def branchit(main,name):
    accesscode=str(input(print("enter your team member access code: ")))
    if accesscode!=main.teammateaccesscode:
        print("unauthorised")
        return


    fs = MainBranch(name,main.admin,main.team)
    s=str(input(print("do you want to clone the main branch? Y/N")))
    if s=='Y':

        print("cloning the main branch")
        for files in main.files:
            fs.add(files)



    print("branch "+name+" created on main branch "+main.name)




    head=main.gethead()
    head.branch=fs
    return fs
def mergeit(main,branch):
    m=main.files
    b=branch.files
    s=str(input(print("do you want to merge the branches as admin? Y/N")))
    if s=='Y':
        accesscode=str(input(print("enter your admin access code: ")))
        if accesscode!=main.admincode:
            return
            print("unauthorised")


        main.commit()
        for file in b:
            if file not in m:
                main.add(file)
        main.commit()
    head=main.gethead()
    head.branch=None
    print("branch "+branch.name+" merged to main "+main.name+" and commited")


class Node:
    def __init__(self, key=None,directory=0):
        self.key = key
        if directory==1:
            self.hash=self.key
        if key is not None and directory!=1:
            self.hash = hash_file(key)

        self.parent = None
        self.rc = None
        self.lc = None
        self.directory=directory


    def hashit(self, rc, lc):
        self.key = None
        self.rc = rc
        self.lc = lc
        if rc==None and lc!=None:
            k=lc.hash
        elif rc!=None and lc==None:
            k=rc.hash
        elif rc is None and lc is None:
            self.hash=None
            return
        else:
            k = rc.hash + lc.hash
        self.hash = hashlib.sha256(k.encode('utf-8')).hexdigest()

class MerkleTree:
    def __init__(self):
        self.root = None
        self.files=[]
  #not used
    def build_tree(self, files):
        if not files:
            return None

        leaf_nodes = [Node(file_name) for file_name in files]
        while len(leaf_nodes) > 1:
            parents = []
            for i in range(0, len(leaf_nodes), 2):
                left_child = leaf_nodes[i]
                right_child = leaf_nodes[i + 1] if i + 1 < len(leaf_nodes) else None
                parent = Node()
                parent.hashit(left_child, right_child)
                left_child.parent = parent
                if right_child:
                    right_child.parent = parent
                parents.append(parent)
            leaf_nodes = parents

        self.root = leaf_nodes[0]

    def add_file(self, file_name,d=0):
        if d==0:
            print("file "+file_name+" added")

        self.files.append(file_name)
        new_node = Node(file_name,d)
        if self.root is None:
            self.root=new_node
            return


        parent = self.root.parent
        # if parent:
        #     new_node.parent = parent
        #     parent.rc = new_node
        #     self.root.parent = None
        #     self.root = new_node
        #     # Recalculate hashes
        #     while parent:
        #         parent.hashit(parent.lc, parent.rc)
        #         parent = parent.parent
        # else:
        parent = Node()
        parent.hashit(self.root, new_node)
        self.root.parent = parent
        new_node.parent = parent
        parent.rc = new_node
        self.root = parent

    def delete_file(self, file_name):
        print("file "+str(file_name)+" removed")

        self.files.remove(file_name)
        node_to_delete = self.find_node(file_name)
        if not node_to_delete:
            print("file "+str(file_name)+" not found")
            return  # File not found

        parent = node_to_delete.parent
        if parent:
            if parent.lc == node_to_delete:
                parent.lc = None
            else:
                parent.rc = None

            # Recalculate hashes
            while parent:
                parent.hashit(parent.lc, parent.rc)
                parent = parent.parent
        else:
            self.root = None

    def find_node(self, file_name):
        if not self.root:
            return None

        q = [self.root]
        while q:
            current = q.pop(0)
            if current.key == file_name:
                return current
            if current.lc:
                q.append(current.lc)
            if current.rc:
                q.append(current.rc)
        return None
    def print_tree(self,root):
        pass

class nodell:
    def __init__(self,value=None):
        self.prev=None
        self.value=value
        self.next=None
        self.branch=None
class DLL:
    def __init__(self):
        self.head=nodell(None)
        self.tail=self.head
        self.length=0

    def insertAtFirst(self,elem):
        if self.length==0:
            self.head.value=elem
        else:
            x=nodell(elem)
            self.head.prev=x
            x.next=self.head
            self.head=x
        self.length+=1



    def deleteAtFirst(self):
        if self.length==0:
            print("UnderFlow")
        elif self.length==1:
            self.head.value=None
            self.length-=1
        else:
            self.length-=1
            x=self.head.next
            x.prev=None
            self.head.next=None
            self.head=x

    def display(self):
        x=self.head

        for i in range(self.length):
            print("commit "+str(i+1)+" root : "+x.value.hash,end=" ")
            x=x.next
        print("")


class MainBranch:
    def __init__(self,name:str,admin,team):
        self.files=[]
        self.admin=admin
        self.team=team
        self.admincode=admin
        self.teammateaccesscode= team
        self.name=name
        print(name+" created")
        self.main=DLL()
        self.historystack=[]
        self.merkle_tree = MerkleTree()
        self.merkle_tree.build_tree([])  # Initialize with an empty list of files
    def showcommit(self):

        self.main.display()
    def add(self, file_name):


        if isinstance(file_name, list):
            print("directory is being added")
            self.directory = MerkleTree()
            for i in file_name:
                self.files.append(file_name)
                self.directory.add_file(i)
            self.merkle_tree.add_file(self.directory.root.hash,1)


        else:
            self.files.append(file_name)
            self.merkle_tree.add_file(file_name)

    def gethead(self):
        return self.main.head

    def showbranch(self):
        return self.main.head.branch
        pass

    def commit(self):

        msg=str(input("comments: "))
        print("branch "+self.name+" commited")
        self.main.insertAtFirst(self.merkle_tree.root)

        self.historystack.append(msg)
        self.merkle_tree = MerkleTree()
    def revoke(self):
        print("branch"+self.name+" revoked")

        self.main.deleteAtFirst()


    def delete(self, file_name):
        self.merkle_tree.delete_file(file_name)

    # def tamper(self,old_file,new_file):
    #     print("old root hash:",self.merkle_tree.root.hash)
    #     self.delete(old_file)
    #     # Add the updated file
    #     self.add(new_file)
    #     print("new root hash:",self.merkle_tree.root.hash)
    def tamper(self, old_file, new_file):
        # Generate a random challenge
        challenge_key = random.randint(1,5)
        print(f"Complete this challenge to proceed: What is {challenge_key} + 3?")

        try:
            response = int(input("Enter your response: "))
        except ValueError:
            print("Invalid response. Tampering aborted.")
            return

        if response != challenge_key + 3:
            print("Incorrect response. Tampering aborted.")
            return

        print("Challenge passed. Please wait...")
        time.sleep(2)

        print("Tampering initiated...")

        if self.merkle_tree.root:
            print("Old root hash:", self.merkle_tree.root.hash)
        else:
            print("Old root hash: None")

        self.delete(old_file)

        self.add(new_file)

        if self.merkle_tree.root:
            print("New root hash:", self.merkle_tree.root.hash)
        else:
            print("New root hash: None")

        print("Tampering completed.")

    def history(self):
        print(self.historystack)
    def print_tree(self):
        self.merkle_tree.print_tree(self.merkle_tree.root)
    def getroot(self):
        print("the root of the current node is : "+str(self.merkle_tree.root.hash))
    def getaccesscodes(self):
        print("team access code: "+str(self.teammateaccesscode)+"\n"+"admin access code: "+self.admincode)

*driver part*

*creating main branch and adding a file*

In [2]:
main = MainBranch("guhan_branch","guhanesh","digital ninjas")
main.add("example1.txt") #/content/example1.txt
main.add("example2.txt")

guhan_branch created
file example1.txt added


FileNotFoundError: [Errno 2] No such file or directory: 'example1.txt'

In [5]:
main.tamper("example1.txt","example2.txt")

Complete this challenge to proceed: What is 1 + 3?
Enter your response: 4
Challenge passed. Please wait...
Tampering initiated...
Old root hash: 0e54850ad3660b6aaac37eed6c8da35644a62169600c401888f6fc3507e5aa87
file example1.txt removed
file example1.txt not found
file example2.txt added
New root hash: b890e555e6f9c02661bc4a6c2793dac2ff2d01e3ddb35c1d3f0d2b62446f1195
Tampering completed.


*adding a directory and commit*

In [6]:
main.add(["example2.txt","example1.txt"])




directory is being added
file example2.txt added
file example1.txt added


*display commits*

In [7]:
main.commit()


comments: commit 1
branch guhan_branch commited


In [8]:
main.showcommit()

commit 1 root : fe59732373d135a8fd3a7ad8e2fede6a9b3bae87f4aba285fc1c85930b913ccf 


*get access codes to branch the main*

In [ ]:
main.getaccesscodes()
abhishekbranch=branchit(main,"abhishek_branch")

team access code: digital ninjas
admin access code: guhanesh
enter your team member access code: 
Nonedigital ninjas
abhishek_branch created
do you want to clone the main branch? Y/N
NoneY
cloning the main branch
file example1.txt added
file example2.txt added
file example2.txt added
directory is being added
file example2.txt added
file example1.txt added
directory is being added
file example2.txt added
file example1.txt added
branch abhishek_branch created on main branch guhan_branch


In [ ]:
main.history()

['first commit added 3 files']


*show the branch*

In [ ]:
b=main.showbranch()
print(b.merkle_tree.root.hash)

6f02c7b0b483665efc2a7633e8c523d9e72d3e17c4df1778d06f26baad778179


In [ ]:
mergeit(main,abhishekbranch)

do you want to merge the branches as admin? Y/N
NoneY
enter your admin access code: 
Noneguhanesh
comments: first commit from branch by admin
branch guhan_branch commited
comments: commit branch in branch
branch guhan_branch commited
branch abhishek_branch merged to main guhan_branch and commited


In [ ]:
main.history()

['first commit added 3 files', 'first commit from branch by admin', 'commit branch in branch']
